In [1]:
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd

from strategy import *

In [38]:
def run_cerebro(dataname,strategy,trading_start,trading_end):
    df = pd.read_csv(dataname+'.csv')
    df['Date']=pd.to_datetime(df['Date'])
    state_column = df.shape[1]-1
        # Instantiate Cerebro engine
    cerebro = bt.Cerebro()

    data = PandasData(
        dataname = df,
        fromdate=trading_start,
        todate=trading_end,
        datetime = 0,
        open = 1,
        high = 2,
        low = 3,
        close = 4,
        state = state_column,
    )

    # Add data to Cerebro
    cerebro.adddata(data)

    # Add strategy to Cerebro
    cerebro.addstrategy(strategy)

    # Default position size
    cerebro.addsizer(bt.sizers.SizerFix, stake=1)

    # Add analytics to Cerebro
    cerebro.addanalyzer(btanalyzers.Returns, _name='Returns')
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='SharpeRatio')
    cerebro.addanalyzer(btanalyzers.VWR, _name='VWR')
    cerebro.addanalyzer(btanalyzers.Calmar, _name='Calmar')
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='DrawDown')
    

    # Run Cerebro Engine
    start_portfolio_value = cerebro.broker.getvalue()

    # cerebro.run()
    thestrats = cerebro.run()
    thestrat = thestrats[0]

    end_portfolio_value = cerebro.broker.getvalue()
    




    pnl = end_portfolio_value - start_portfolio_value
    total_compound_return = thestrat.analyzers.Returns.get_analysis()['rtot']
    sharpe_ratio = thestrat.analyzers.SharpeRatio.get_analysis()['sharperatio']
    variability_weighted_return = thestrat.analyzers.VWR.get_analysis()['vwr']
    calmar_ratio = thestrat.analyzers.Calmar.get_analysis()
    max_drawdown = thestrat.analyzers.DrawDown.get_analysis()['max']['drawdown']
    max_drawdown_len = thestrat.analyzers.DrawDown.get_analysis()['max']['len']

    print(f'Starting Portfolio Value: {start_portfolio_value:2f}')
    print(f'Final Portfolio Value: {end_portfolio_value:2f}')
    print(f'PnL: {pnl:.2f}\n')
    return pnl, total_compound_return, sharpe_ratio, variability_weighted_return,max_drawdown,max_drawdown_len


In [39]:
datanames = [
    # # nn
    # 'nn_y1_2003_LSTM', 'nn_y1_2003_BI-LSTM', 'nn_y1_2010_LSTM', 'nn_y1_2010_BI-LSTM',
    # 'nn_y1_2015_LSTM', 'nn_y1_2015_BI-LSTM', 'nn_y2_2003_LSTM', 'nn_y2_2003_BI-LSTM',
    # 'nn_y2_2010_LSTM', 'nn_y2_2010_BI-LSTM', 'nn_y2_2015_LSTM', 'nn_y2_2015_BI-LSTM',
    # 'nn_y5_2003_LSTM', 'nn_y5_2003_BI-LSTM', 'nn_y5_2010_LSTM', 'nn_y5_2010_BI-LSTM',
    # 'nn_y5_2015_LSTM', 'nn_y5_2015_BI-LSTM', 'nn_gm_2003_LSTM', 'nn_gm_2003_BI-LSTM',
    # 'nn_gm_2010_LSTM', 'nn_gm_2010_BI-LSTM', 'nn_gm_2015_LSTM', 'nn_gm_2015_BI-LSTM',
    # 'nn_hm_2003_LSTM', 'nn_hm_2003_BI-LSTM', 'nn_hm_2010_LSTM', 'nn_hm_2010_BI-LSTM',
    # 'nn_hm_2015_LSTM', 'nn_hm_2015_BI-LSTM',
    # # hmm
    # 'gmm_1_2003','gmm_1_2010','gmm_1_2015',
    # 'hmm_1_2003','hmm_1_2010','hmm_1_2015',
    # 'hmm_7_2003','hmm_7_2010','hmm_7_2015'
    # # rf
    # 'rf_y1_2003', 'rf_y1_2010', 'rf_y1_2015', 'rf_y2_2003',
    # 'rf_y2_2010', 'rf_y2_2015', 'rf_y5_2003', 'rf_y5_2010',
    # 'rf_y5_2015', 'rf_gm_2003', 'rf_gm_2010', 'rf_gm_2015',
    # 'rf_hm_2003', 'rf_hm_2010', 'rf_hm_2015',
    # xg
    # 'xg_y1_2003', 'xg_y1_2010', 'xg_y1_2015', 'xg_y2_2003',
    # 'xg_y2_2010', 'xg_y2_2015', 'xg_y5_2003', 'xg_y5_2010',
    # 'xg_y5_2015', 'xg_gm_2003', 'xg_gm_2010', 'xg_gm_2015',
    # 'xg_hm_2003', 'xg_hm_2010', 'xg_hm_2015',
    # plain
    'y1_2003','y2_2003','y5_2003','hmm_1_labeled_2003','gmm_1_labeled_2003',
    



]
strategy_list = [MarketStatus,BuyAndHold]
trading_start_list = [datetime.datetime(2018, 1, 2)]
trading_end_list = [datetime.datetime(2021, 12, 30)]
pnl_result = []
pnl = 0
for dataname in datanames:
    for trading_start in trading_start_list:
        for trading_end in trading_end_list:
            for strategy in strategy_list:
                pnl, total_compound_return, sharpe_ratio, variability_weighted_return,max_drawdown,max_drawdown_len = run_cerebro(dataname,strategy,trading_start,trading_end)
                result_dic = {'dataname' : dataname,
                                'trading_start': trading_start,
                                'trading_end': trading_end,
                                'strategy':strategy,
                                'pnl' : pnl, 
                                'total_compound_return' : total_compound_return,
                                'sharpe_ratio': sharpe_ratio,
                                'variability_weighted_return': variability_weighted_return,
                                'max_drawdown': max_drawdown,
                                'max_drawdown_len': max_drawdown_len
                }
                pnl_result.append(result_dic)
pnl_result

Starting Portfolio Value: 10000.000000
Final Portfolio Value: 11044.569948
PnL: 1044.57

2018-01-02 BUY CREATE 1595.869995
2018-01-03 BUY EXECUTED, 1595.84
Starting Portfolio Value: 10000.000000
Final Portfolio Value: 11200.069946
PnL: 1200.07

Starting Portfolio Value: 10000.000000
Final Portfolio Value: 8769.200689
PnL: -1230.80

2018-01-02 BUY CREATE 1595.869995
2018-01-03 BUY EXECUTED, 1595.84
Starting Portfolio Value: 10000.000000
Final Portfolio Value: 11200.069946
PnL: 1200.07

Starting Portfolio Value: 10000.000000
Final Portfolio Value: 10986.729605
PnL: 986.73

2018-01-02 BUY CREATE 1595.869995
2018-01-03 BUY EXECUTED, 1595.84
Starting Portfolio Value: 10000.000000
Final Portfolio Value: 11200.069946
PnL: 1200.07

Starting Portfolio Value: 10000.000000
Final Portfolio Value: 12759.541631
PnL: 2759.54

2018-01-02 BUY CREATE 1595.869995
2018-01-03 BUY EXECUTED, 1595.84
Starting Portfolio Value: 10000.000000
Final Portfolio Value: 11200.069946
PnL: 1200.07

Starting Portfolio Va

[{'dataname': 'y1_2003',
  'trading_start': datetime.datetime(2018, 1, 2, 0, 0),
  'trading_end': datetime.datetime(2021, 12, 30, 0, 0),
  'strategy': strategy.MarketStatus,
  'pnl': 1044.5699479999967,
  'total_compound_return': 0.09935380676870487,
  'sharpe_ratio': 0.7449190187575274,
  'variability_weighted_return': 1.4944177375275298,
  'max_drawdown': 4.206037743390235,
  'max_drawdown_len': 228},
 {'dataname': 'y1_2003',
  'trading_start': datetime.datetime(2018, 1, 2, 0, 0),
  'trading_end': datetime.datetime(2021, 12, 30, 0, 0),
  'strategy': strategy.BuyAndHold,
  'pnl': 1200.0699459999996,
  'total_compound_return': 0.11333493046607353,
  'sharpe_ratio': 0.772832856118916,
  'variability_weighted_return': 1.675995079544723,
  'max_drawdown': 6.701885060973381,
  'max_drawdown_len': 172},
 {'dataname': 'y2_2003',
  'trading_start': datetime.datetime(2018, 1, 2, 0, 0),
  'trading_end': datetime.datetime(2021, 12, 30, 0, 0),
  'strategy': strategy.MarketStatus,
  'pnl': -1230.7

In [40]:
pnl_result_df = pd.DataFrame(pnl_result)
pnl_result_df.sort_values(by='pnl',ascending=False)
pnl_result_df.to_excel('0 plain_final_results.xlsx')

In [ ]:
visual=df.loc["2018-01-02":"2021-12-31"]
visual["State"]=hidden_states
colors= {0:'red', 1:'green', 2:"blue"}
plt.figure(figsize=(20,8))
plt.scatter(visual.index, visual['Close'], c=visual['State'].map(colors))

NameError: name 'df' is not defined